In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://www.imdb.com/list/ls056092300/'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

movies = soup.find_all('div', class_='lister-item mode-detail')

movie_names = []
movie_ratings = []
movie_years = []

for movie in movies:
    name = movie.h3.a.text
    movie_names.append(name)
    
    rating = movie.find('span', class_='ipl-rating-star__rating').text
    movie_ratings.append(float(rating))
    
    year = movie.find('span', class_='lister-item-year text-muted unbold').text
    year = year.strip('()').split()[0]  # Clean the year
    movie_years.append(int(year))

# DataFrame
movies_df = pd.DataFrame({'Name': movie_names,'Rating': movie_ratings,'Year of Release': movie_years})

print(movies_df)

Empty DataFrame
Columns: [Name, Rating, Year of Release]
Index: []


In [11]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://www.patreon.com/coreyms'

# Request to fetch the content of the webpage
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36'}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    
    headings = []
    dates = []
    contents = []
    video_likes = []
    
    posts = soup.find_all('div', class_='sc-16b9v2b-0')
    
    for post in posts:
        # Extract heading
        heading_tag = post.find('h2')
        heading = heading_tag.text.strip() if heading_tag else "No Heading"
        headings.append(heading)
        
        # Extract date
        date_tag = post.find('time')
        date = date_tag.text.strip() if date_tag else "No Date"
        dates.append(date)
        
        # Extract content
        content_tag = post.find('div', class_='sc-12p78pa-0')
        content = content_tag.text.strip() if content_tag else "No Content"
        contents.append(content)
        
        # Extract YouTube link
        video_tag = post.find('a', href=True)
        video_link = video_tag['href'] if video_tag and 'youtube' in video_tag['href'] else "No Video Link"
        if video_link != "No Video Link":
            video_likes.append("Likes Not Scraped")
        else:
            video_likes.append("No Likes")
    
# DataFrame
    posts_df = pd.DataFrame({'Heading': headings,'Date': dates,'Content': contents,'Video Likes': video_likes})
    
    print(posts_df)
else:
    print("Failed to retrieve the page.")

Empty DataFrame
Columns: [Heading, Date, Content, Video Likes]
Index: []


In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define base URL and headers
base_url = "https://www.nobroker.in/property/sale/"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36'}

# List of localities
localities = ['Indira-Nagar', 'Jayanagar', 'Rajaji-Nagar']

# Lists to store scraped data
titles = []
locations = []
areas = []
emis = []
prices = []

# Function to scrape data for a given locality
def scrape_locality_data(locality):
    url = base_url + locality.replace(' ', '-')
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract the details of houses
        property_listings = soup.find_all('div', class_='card__details-container')

        for property in property_listings:
            # Extract the house title
            title_tag = property.find('span', class_='card__title')
            title = title_tag.text if title_tag else 'No Title'
            
            # Extract location
            location_tag = property.find('div', class_='card__locality')
            location = location_tag.text if location_tag else 'No Location'
            
            # Extract area
            area_tag = property.find('div', class_='card__area')
            area = area_tag.text if area_tag else 'No Area'
            
            # Extract price
            price_tag = property.find('div', class_='card__price')
            price = price_tag.text.strip() if price_tag else 'No Price'
            
            # EMI - Not provided directly on the page, can calculate or placeholder
            emi = "Not Available"
            
            # Append details to lists
            titles.append(title)
            locations.append(location)
            areas.append(area)
            prices.append(price)
            emis.append(emi)
    else:
        print(f"Failed to retrieve data for {locality}")

# Scrape data for each locality
for locality in localities:
    scrape_locality_data(locality)

# Create a DataFrame from the scraped data
house_data = pd.DataFrame({
    'Title': titles,
    'Location': locations,
    'Area': areas,
    'Price': prices,
    'EMI': emis
})

# Display the DataFrame
print(house_data)

# Save data to CSV
house_data.to_csv('nobroker_house_details.csv', index=False)

Empty DataFrame
Columns: [Title, Location, Area, Price, EMI]
Index: []


In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the Bewakoof bestseller page
url = "https://www.bewakoof.com/bestseller?sort=popular"

# Send a request to fetch the content of the webpage
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36'}
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the containers for the first 10 products
    product_containers = soup.find_all('div', class_='productCardBox', limit=10)
    
    # Lists to store product details
    product_names = []
    product_prices = []
    product_image_urls = []
    
    # Loop through each product container and extract details
    for product in product_containers:
        # Extract product name
        name_tag = product.find('h3')
        product_name = name_tag.text.strip() if name_tag else "No Name"
        product_names.append(product_name)
        
        # Extract product price
        price_tag = product.find('span', class_='discountedPriceText')
        product_price = price_tag.text.strip() if price_tag else "No Price"
        product_prices.append(product_price)
        
        # Extract product image URL
        image_tag = product.find('img')
        product_image_url = image_tag['src'] if image_tag else "No Image URL"
        product_image_urls.append(product_image_url)
    
    # DataFrame to store the scraped data
    products_df = pd.DataFrame({'Product Name': product_names,'Price': product_prices,'Image URL': product_image_urls})
    
    print(products_df)
    
    # Save the data to a CSV file if needed
    # products_df.to_csv('bewakoof_bestsellers.csv', index=False)
    
else:
    print("Failed to retrieve the page.")


                     Product Name     Price  \
0                   bewakoof x dc  No Price   
1  bewakoof x house of the dragon  No Price   
2          bewakoof x tom & jerry  No Price   
3                       Bewakoof®  No Price   
4                       Bewakoof®  No Price   
5              bewakoof x peanuts  No Price   
6                       Bewakoof®  No Price   
7                       Bewakoof®  No Price   
8          bewakoof x tom & jerry  No Price   
9                       Bewakoof®  No Price   

                                           Image URL  
0  https://images.bewakoof.com/t640/men-s-black-a...  
1  https://images.bewakoof.com/t640/men-s-black-h...  
2  https://images.bewakoof.com/t640/women-s-blue-...  
3  https://images.bewakoof.com/t640/women-aop-ove...  
4  https://images.bewakoof.com/t640/men-s-black-w...  
5  https://images.bewakoof.com/t640/women-s-orang...  
6  https://images.bewakoof.com/t640/men-s-black-l...  
7  https://images.bewakoof.com/t640/men-s-

In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.cnbc.com/world/?region=world"

# Send a request to fetch the content of the webpage
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36'}
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Lists to store scraped data
    headings = []
    dates = []
    links = []
    
    # Find the news articles container
    articles = soup.find_all('div', class_='Card-titleContainer')
    
    for article in articles:
        # Extract news heading
        heading_tag = article.find('a')
        heading = heading_tag.text.strip() if heading_tag else "No Heading"
        headings.append(heading)
        
        # Extract news link
        news_link = heading_tag['href'] if heading_tag else "No Link"
        links.append(news_link)
        
        # Extract publication date if available (CNBC doesn't always show dates in this view)
        date_tag = article.find_next('time')  # Find next time element
        date = date_tag.text.strip() if date_tag else "No Date"
        dates.append(date)

    # Create a DataFrame
    news_df = pd.DataFrame({'Heading': headings,'Date': dates,'Link': links})
    
    print(news_df)
    
    # Save to CSV file
    # news_df.to_csv('cnbc_world_news.csv', index=False)
    
else:
    print("Failed to retrieve the page.")

                                              Heading     Date  \
0   Stocks making the biggest moves midday: JetBlu...  No Date   
1   JPMorgan top economist says the Fed should cut...  No Date   
2   Dow falls more than 200 points as growth fears...  No Date   
3   Friday's jobs report for August is going to be...  No Date   
4   'Get Britain building again': New UK finance c...  No Date   
5   Britain's Labour pulled off a thumping electio...  No Date   
6   UK's Labour Party secures landslide victory in...  No Date   
7   Labour does not have much headroom in terms of...  No Date   
8   Outgoing UK PM Rishi Sunak to step down as Con...  No Date   
9   Water shortages are likely brewing future wars...  No Date   
10  Sweden's Volvo Cars scraps plan to sell only e...  No Date   
11  ESG investors appear to be getting much more c...  No Date   
12  Finland will soon bury spent nuclear fuel in t...  No Date   
13  This water treatment startup is already a unic...  No Date   
14  Passen

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the KEAI Publishing Most Downloaded Articles page
url = "https://www.keaipublishing.com/en/journals/artificial-intelligence-in-agriculture/most-downloaded/articles/"

# Send a request to fetch the content of the webpage
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36'}
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Lists to store scraped data
    titles = []
    dates = []
    authors = []
    
    # Find the containers for the articles
    articles = soup.find_all('article', class_='article')
    
    for article in articles:
        # Extract paper title
        title_tag = article.find('h2', class_='title')
        title = title_tag.text.strip() if title_tag else "No Title"
        titles.append(title)
        
        # Extract publication date
        date_tag = article.find('time')
        date = date_tag.text.strip() if date_tag else "No Date"
        dates.append(date)
        
        # Extract author names
        author_tags = article.find_all('a', class_='author')
        authors_list = [author.text.strip() for author in author_tags]
        authors.append(", ".join(authors_list) if authors_list else "No Authors")

    # Create a DataFrame from the collected data
    articles_df = pd.DataFrame({'Paper Title': titles,'Date': dates,'Author': authors})
    
    # Display the DataFrame
    print(articles_df)
    
    # Save to CSV file
    # articles_df.to_csv('keaipublishing_most_downloaded_articles.csv', index=False)
    
else:
    print("Failed to retrieve the page.")

Failed to retrieve the page.
